# Análise de sentimentos no Twitter - Eleições 2022

Este código, desenvolvido em Python, extrai os 100 tweets mais recentes com a API do Twitter pelos temas: "#Eleições2022 -#Lula". 

Também, realiza tarefas como: 
* Agrupar todos em dataframes
* Gerar uma imagem com as palavras mais presentes nos tweets (word cloud)
* Traduzir os tweets para inglês
* Adicionar as colunas subjetividade, polaridade no dataframe
* Fazer a análise do texto em base da polaridade e classificá-lo como sendo: Positivo, Negativo ou Neutro na coluna de análise do Dataframe.
* Mostrar a polaridade e subjetividade num gráfico de dispersão
* Mostrar quantidade de tweets de acordo com o seu sentimento em gráfico de barras

In [ ]:
# installa dependencias

# !pip install tweepy --quiet
# !pip install pandas --quiet
# !pip install wordcloud
# !pip install --user -U nltk

In [ ]:
# importa módulos
import tweepy
import pandas as pd
import re
import nltk
import requests
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
plt.style.use('fivethirtyeight')
nltk.download('stopwords')

In [ ]:
# lê as credenciais de login do Twitter
with open("login.txt", "r") as file:
    consumer_key=file.readline().strip("\n")
    consumer_secret=file.readline().strip("\n")
    bearer_token=file.readline().strip("\n")
    access_token=file.readline().strip("\n")
    access_token_secret=file.readline().strip("\n")

In [ ]:
# faz autenticação
client = tweepy.Client(
    bearer_token
)

In [ ]:
# pesquisa por tweets por palavra-chave: "#Eleições2022 -#Lula"
query = "#Eleições2022 -#Lula"
response = client.search_recent_tweets(query, max_results=100)
tweets = response.data

for tweet in tweets:
    print(tweet.text)

In [ ]:
# Agrupa tweets por dataframe
tweets_list = [[tweet.text] for tweet in tweets]
df = pd.DataFrame(tweets_list, columns=["Texto"])
pd.options.display.max_colwidth = 130
df.head()

In [ ]:
# Limpando o texto
def cleanText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # removed mentions
    text = re.sub(r'#', '', text) # remove the symbol '#'
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text) # remove the hyper link
    return text

# aplicando a função no DataFrame
df['Texto'] = df['Texto'].apply(cleanText)

# mostra o Dataframe
df

In [ ]:
# Gera wordcloud dos tweets

# Agrupando tudo em uma única string com espaços
string = ''.join([twts for twts in df['Texto']])

# Ignora classes de palavras
stopwords = set(STOPWORDS)
stopwords_nltk = nltk.corpus.stopwords.words('portuguese') 
stopwords_manual = ["quint", "anda", "vídeo", "amg", "onde", "meio", "con", "todo", "apoio", "chama", "compartilhado", "cargo", "TikTok", "Eleicoes2022", "hora", "insta", "rede", "ver", "conteúdo", "perfil", "muito", "Quando", "derrotar", "público","eleições","Eleições2022A", "prestação", "tamb", "turno", "conta", "la", "declara", "livre", "Porto", "Alegre","AGORA", "single", "anormal", "votar", "vota","aconteceu", "atualizada", "algo", "Jr", "acontece", "terá", "_Mordacious_", "passe", "pressão", "Já", "detalhes", "THREAD", "minha", "TV", "maior", "mesmo", "Atenção", "desde", "há", "número", "presidente", "ENQUETE", "mostra","será", "queremos","vocês", "conferir","meu", "y", "pela", "seus", "comentamos", "você", "PREDICTION", "como", "nosso", "tá", "vão", "dia", "instituto", "fez", "Eleições2022", "RT", "por", "de", "f", "w", "pelo", "Brazilian", "dar", "pois", "nesta", "em", "um", "da", "ser", "aqui", "vou", "dos", "quem", "tem", "NEW", "do", "não", "election", "t", "foi", "para", "made", "https", "ahead", "Na", "S", "ao", "que", "co", "coming", "ad", "O", "é", "Mais", "se", "e", "das", "depois", "dessa", "ontem", "uma", "tira", "site", "diz", "tempo", "deveria", "certo", "até", "dúvida", "importo", "qual", "Só", "sabendo", "estão", "gesto", "nos", "ocorreu", "Fiquei", "rimou", "chorar", "essa", "pra", "os", "qual", "ponto", "escolher", "mas", "vamos", "AR", "pode", "junto", "demais", "chamado", "parecer", "ainda", "dias", "nas", "sobre", "está", "faz", "Leia", "pagou", "gente", "genérico", "durante", "apenas", "São", "à", "às", "entre", "vai", "busco", "fazer", "eu", "todos", "seu", "primeiro"]
stopwords.update(stopwords_nltk + stopwords_manual)


# Cria uma imagem com as palavras mais presentes no texto
wordcloud = WordCloud(width=1600, stopwords=stopwords, height=800, max_font_size=200, max_words=30, collocations=False, background_color='black').generate(string)
plt.figure(figsize=(40,30))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")

# realiza download do plot
plt.savefig("Twitter-WordCloud.png")

plt.show()

In [ ]:
# Realiza tradução dos textos em 'pt-BR' para 'en'
# Para que a classe TextBlob possa fazer a análise
# Obs: a tradução demora, já que realiza uma requisição por vez
def translate_to_en(text):
    url = f"http://translate.google.com/m?tl=pt-br&sl=en&q={text}"
    expr = r'(?s)class="(?:t0|result-container)">(.*?)<'
    content = requests.get(url).content
    data = content.decode('utf-8')
    return re.findall(expr, data)[0]
    
# aplica a função no dataFrame
df['Texto'] = df['Texto'].apply(translate_to_en)

# mostra o dataframe
df

In [ ]:
# pega subjectividade
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# pega a polaridade
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# cria uma função para computar se a análise é negativa, positiva ou neutra
def getAnalysis(score):
    if score < 0:
        return "Negativo"
    elif score == 0:
        return "Neutro"
    else:
        return "Positivo"


# cria colunas de subjetividade, polaridade e análise no dataframe
df['Subjetividade'] = df['Texto'].apply(getSubjectivity)
df['Polaridade'] = df['Texto'].apply(getPolarity)
df["Análise"] = df['Polaridade'].apply(getAnalysis)

# mostra dataframe
df

In [ ]:
# mostra todos os tweets potencialmente positivos
sortedDF = df.sort_values(by=['Polaridade'])

for i in range(sortedDF.shape[0]):
    if sortedDF["Análise"][i] == "Positivo":
        print(f"{i + 1}) {sortedDF['Texto'][i]}\n")

In [ ]:
# mostra todos os tweets potencialmente negativos
sortedDF = df.sort_values(by=['Polaridade'], ascending='False')

for i in range(sortedDF.shape[0]):
    if sortedDF["Análise"][i] == "Negativo":
        print(f"{i + 1}) {sortedDF['Texto'][i]}\n")

In [ ]:
# monta gráfico de dispersão mostrando a polaridade e subjetividade
plt.figure(figsize=(8, 6))
for i in range(df.shape[0]):
    plt.scatter(df['Polaridade'][i], df['Subjetividade'][i], color='Blue')

plt.title("Subjetividade X Polaridade")
plt.xlabel("Polaridade")
plt.ylabel("Subjetividade")
plt.show()

In [ ]:
# Pega porcentagem dos tweets pelo sentimento
def sentiment_percent(sentiment):
    ptweets = df[df.Análise == sentiment]
    ptweets = ptweets["Texto"]

    return round(ptweets.shape[0] / df.shape[0] * 100, 1) 

text = f"""\
tweets positivos: {sentiment_percent("Positivo")}%
tweets negativos: {sentiment_percent("Negativo")}%
tweets neutros: {sentiment_percent("Neutro")}%\
"""
print(text)

In [ ]:
# traça e visualiza o gráfico de barras da quantidade de tweets de acordo com o seu sentimento

plt.title("Análise de sentimentos - Eleições 2022")
plt.xlabel("Sentimentos")
plt.ylabel("Quantidade")
df["Análise"].value_counts().plot(kind='bar')
plt.show()